In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.regularizers import l2
import pickle
import numpy as np
import datetime
import os
now_datetime  = datetime.datetime.now()
NAME = f"#Face_spoofing300x2_{now_datetime.day:02d}{now_datetime.month:02d}{now_datetime.year}_{now_datetime.hour:02d}{now_datetime.minute:02d}"
dir_pickle = "database_serilased"
dir_models_save = "models"
person = "001"

Using TensorFlow backend.


In [2]:
def create_model(person, pickle_path):
    pickle_in = open(os.path.join(pickle_path,f"X{person}.pickle"),"rb")
    X = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}.pickle"),"rb")
    y = pickle.load(pickle_in)
    pickle_in.close()
    X = X/255.0
    model = Sequential()
    #first conv layer
    model.add(layers.Conv2D(68,(3,3),input_shape=X.shape[1:]))
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    #second conv layer
    model.add(layers.Conv2D(128,(3,3)))
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPool2D(pool_size=(3,3)))
    #third conv layer
    model.add(layers.Conv2D(128,(3,3)))
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    #forth conv layer
    model.add(layers.Conv2D(256,(3,3)))
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPool2D(pool_size=(3,3)))
    
    model.add(layers.Flatten())
    #first dense layer
    model.add(layers.Dropout(0.10))
    model.add(layers.Dense(128,activation="relu",kernel_regularizer=l2(0.002)))
    #second dense layer
    model.add(layers.Dropout(0.15))             
    model.add(layers.Dense(200,activation="relu",kernel_regularizer=l2(0.002)))
    #third dense layer           
    model.add(layers.Dense(200,activation="relu",kernel_regularizer=l2(0.002)))
    #final dense layer
    model.add(layers.Dense(1,activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics= ["accuracy"])
    
    return X, y, model

In [3]:
X, y, model = create_model(person,os.path.join(dir_pickle,person))

W1004 18:36:22.171079 10176 deprecation.py:506] From E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1004 18:36:22.502225 10176 deprecation.py:323] From E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
model.fit(X,y,batch_size=9,validation_split=0.17,epochs=19,shuffle=False)

Train on 2942 samples, validate on 603 samples
Epoch 1/19
2942/2942 [==============================] - 54s 19ms/sample - loss: 0.3865 - acc: 0.9473 - val_loss: 0.1309 - val_acc: 1.0000
Epoch 2/19
2942/2942 [==============================] - 59s 20ms/sample - loss: 0.1373 - acc: 0.9939 - val_loss: 0.1525 - val_acc: 0.9834
Epoch 3/19
2942/2942 [==============================] - 65s 22ms/sample - loss: 0.1651 - acc: 0.9857 - val_loss: 0.0929 - val_acc: 1.0000
Epoch 4/19
2942/2942 [==============================] - 64s 22ms/sample - loss: 0.0749 - acc: 1.0000 - val_loss: 0.0557 - val_acc: 1.0000
Epoch 5/19
2942/2942 [==============================] - 64s 22ms/sample - loss: 0.0469 - acc: 1.0000 - val_loss: 0.0374 - val_acc: 1.0000
Epoch 6/19
2942/2942 [==============================] - 64s 22ms/sample - loss: 0.0313 - acc: 1.0000 - val_loss: 0.0253 - val_acc: 1.0000
Epoch 7/19
2942/2942 [==============================] - 52s 18ms/sample - loss: 0.0325 - acc: 0.9986 - val_loss: 0.0234 - val

In [7]:
import keras
model.save(os.path.join(dir_models_save,f"{person}{NAME}[68,128,128,256][128,200,200].h5"))
keras.backend.clear_session()

ValueError: Fetch argument <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 68) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("conv2d/kernel/Read/ReadVariableOp:0", shape=(3, 3, 3, 68), dtype=float32) is not an element of this graph.)

In [6]:
pickle_in = open(os.path.join(dir_pickle,"001","X001Test.pickle"),"rb")
Xtest = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open(os.path.join(dir_pickle,"001","y001Test.pickle"),"rb")
ytest = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open(os.path.join(dir_pickle,"001","y001Test.pickle"),"rb")
ytest = pickle.load(pickle_in)
pickle_in.close()

FileNotFoundError: [Errno 2] No such file or directory: 'database_serilased\\001\\X001Test.pickle'

In [12]:
 results = smodel.evaluate(Xtest,ytest)

220/220 [==============================] - 2s 11ms/sample - loss: 71.2231 - acc: 0.8545
